In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# from wordcloud import WordCloud

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
data = pd.read_csv("Montreal Crime Data.csv")

In [31]:
data.head()

,category,date,postal_code,city,neighbourhood,year,count,longitude,latitude
0,Motor vehicle theft,2018-09-13,H1Z 1S9,MONTREAL,Saint-Michel,2018,1,-73.626,45.567
1,Motor vehicle theft,2018-04-30,H1Z 1S9,MONTREAL,Saint-Michel,2018,1,-73.626,45.567
2,Home Invasion,2018-01-10,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569
3,Mischief,2018-11-12,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569
4,Mischief,2018-08-15,H1Z 2V6,MONTREAL,Saint-Michel,2018,1,-73.629,45.569


# Preprocessing

#### Removing unwanted column

In [6]:
data.drop("Unnamed: 0", axis = 1, inplace = True)

In [7]:
data.columns

Index(['category', 'date', 'postal_code', 'city', 'neighbourhood', 'year',
       'count', 'longitude', 'latitude'],
      dtype='object')

#### Meta information of data

In [15]:
print(f"Shape of data is: {data.shape}")

Shape of data is: (136642, 9)


In [16]:
pd.DataFrame(data.dtypes, columns=['Datatype']).rename_axis('Columns')

,Datatype
Columns,
category,object
date,datetime64[ns]
postal_code,object
city,object
neighbourhood,object
year,int64
count,int64
longitude,float64
latitude,float64


## Data cleaning

#### Converting date feature to datatime format

In [19]:
data['date'] = pd.to_datetime(data['date'])

In [20]:
datatypes = pd.DataFrame(data.dtypes, columns=['Datatype']).rename_axis('Columns')
datatypes

,Datatype
Columns,
category,object
date,datetime64[ns]
postal_code,object
city,object
neighbourhood,object
year,int64
count,int64
longitude,float64
latitude,float64


#### Checking for NaN values

In [30]:
pd.DataFrame(data.isnull().sum(), columns=['Nb of missing values']).rename_axis('Feature')

,Nb of missing values
Feature,
category,0
date,0
postal_code,0
city,0
neighbourhood,0
year,0
count,0
longitude,0
latitude,0


## Visualization

#### Top 10 Cities where most Crime are reported